In [1]:
# Load relevant packages
import numpy  as np
import pandas as pd

# Load In:
## 2018-2019 School Quality Report (SQR) :  Summary & Student Achievement Data
Source:
https://infohub.nyced.org/reports/school-quality/school-quality-reports-and-resources/school-quality-report-citywide-data

In [2]:
sqr_xlsx = pd.ExcelFile('data/201819_hs_sqr_results.xlsx')

# read in quality report excel file - summary 
sqr_summary = pd.read_excel(sqr_xlsx,'Summary', header=3, usecols='D:G,AF:AI,AK,AM:AW')
sqr_summary.drop(0, inplace = True)
# read in quality report excel file - student achievement
sqr_sa = pd.read_excel(sqr_xlsx,'Student Achievement', header=3, usecols='D:F,BF:BG,BW,CU,EZ:FA,FH:FI,FW')
sqr_sa.drop(0, inplace = True)

In [3]:
# merge summary & student achievement from the same quality report excel file
demograph_df = pd.merge(sqr_summary,sqr_sa, on=['DBN','School Name'], how='inner')

print('sqr_summary dimensions: {}'.format(sqr_summary.shape))
print('sqr_sa dimensions: {}'.format(sqr_sa.shape))
print('demograph_df dimensions: {}'.format(demograph_df.shape))

sqr_summary dimensions: (486, 20)
sqr_sa dimensions: (486, 12)
demograph_df dimensions: (486, 30)


In [4]:
# drop unnecessary columns
demograph_df = demograph_df.drop(columns = ['School Type_y'])
demograph_df.rename(columns={"School Type_x":"School Type"}, inplace = True)

demograph_df.head(1)

,DBN,School Name,School Type,Enrollment,Average Grade 8 English Proficiency,Average Grade 8 Math Proficiency,Percent English Language Learners,Percent Students with Disabilities,Economic Need Index,Percent in Temp Housing,...,Teacher Attendance Rate,N count - 4-Year Graduation Rate - All Students,Metric Value - 4-Year Graduation Rate - All Students,Metric Value - Average Regents Score - English (Common Core),Metric Value - Average Regents Score - Algebra I (Common Core),N count - College and Career Preparatory Course Index,Metric Value - College and Career Preparatory Course Index,N count - 4-Year College Readiness Index,Metric Value - 4-Year College Readiness Index,Percentage of Students with 90%+ Attendance
0,01M292,Orchard Collegiate Academy,High School,197.0,2.701,2.325,0.117,0.289,0.871,0.183,...,0.97,28,0.893,81.1,68.9,28,0.196,28,0.5,0.711


# Load In:
## 2018-2019 School Funding Transparency Form
Source: https://infohub.nyced.org/reports/financial/financial-data-and-reports/new-york-state-school-funding-transparency-forms

In [5]:
# read in financial data
financial_xlsx = pd.ExcelFile('data/NewYorkCitySchoolTransparency201819 A-E web.xlsx')
financial_df = pd.read_excel(financial_xlsx,'Part-C',header=6, usecols = ['School Name','Local School Code',
                       'State & Local Funding',
                       'Federal \nFunding',
                       'Total Funding Source by School'])
# convert to in Millions
financial_df['Total Funding Source by School']= financial_df['Total Funding Source by School'].div(1000000)
financial_df['State & Local Funding']= financial_df['State & Local Funding'].div(1000000)
financial_df['Federal \nFunding']= financial_df['Federal \nFunding'].div(1000000)

# change local school code to BN and renaming columns
financial_df.rename(columns={"Local School Code":"BN",
                             "State & Local Funding":"State & Local Funding (M)",
                             "Federal \nFunding":"Federal Funding (M)", 
                             "Total Funding Source by School":"Total Funding (M)" 
                            }, inplace = True)
financial_df.head(2)

,School Name,BN,State & Local Funding (M),Federal Funding (M),Total Funding (M)
0,P.S. 001 The Bergen,K001,19.124833,1.702345,20.827178
1,Parkside Preparatory Academy,K002,7.642887,0.552985,8.195872


# Merge Date Frames
>We merged the **School Transparency Dataset** from the NYC DOE via the unique school code idenitifer (District Borough Number, DBN) to create a more comphrensive dataframe for our project analysis. School funding may provide more context to measure the relationship of how much a school receives compared to their high performance.

>df = demograph_df + financial_df

In [6]:
# take last 4 digits of DBN to merge with local school code
demograph_df['BN'] = demograph_df['DBN'].str[-4:]

# merging Financial data and quality report
df = pd.merge(demograph_df, financial_df, on='BN', how='left')

# dropping duplicate columns
del df['School Name_y']
df.rename(columns={'School Name_x': 'School Name'}, inplace = True)

print('df dimensions: {}'.format(df.shape))

df dimensions: (486, 33)


In [7]:
# create function to assign borough based on DBN
def borough_finder (value):
    if value[2] == 'M' :
      return 'Manhattan'
    if value[2] == 'K' :
      return 'Brooklyn'
    if value[2] == 'Q' :
      return 'Queens'
    if value[2] == 'R' :
      return 'Staten Island'
    if value[2] == 'X' :
      return 'Bronx'
    return 'Other'
# create column for borough
df['Borough'] = df['DBN'].apply (lambda value: borough_finder(value))

In [8]:
# create column for number of students with an economic need
df['N count - Economic Need Index'] = df['Economic Need Index'] * df['Enrollment']

df[['N count - Economic Need Index','Enrollment','Economic Need Index']].head(2)

,N count - Economic Need Index,Enrollment,Economic Need Index
0,171.587,197.0,0.871
1,400.890,483.0,0.830


In [9]:
# create column for # of students of each race
df['White'] = df['Enrollment']*df['Percent White']
df['Hispanic'] = df['Enrollment']*df['Percent Hispanic']
df['Black'] = df['Enrollment']*df['Percent Black']
df['Asian'] = df['Enrollment']*df['Percent Asian']

# Full Data Frame
This dataset shared the Economic Needs Index (ENI), which is an index created by NYC DOE to measure the economic need based on the following 4 social situations of the students and their families: 

1. Be eligible for Human Resources Administration (HRA) benefits (Public Assistance); 
2. Have lived in temporary housing in the last four years; 
3. Have entered the NYC DOE for the first time within the last four years, is enrolled in high school 
4. English Language Learner (has a home language other than English)

Thus, a high ENI could be a factor that influence high performance in New York City public high schools due to the social conditions of the students and families. 

The school’s Economic Need Index is the average of its students’ Economic Need Values

In [10]:
df.head(1)

,DBN,School Name,School Type,Enrollment,Average Grade 8 English Proficiency,Average Grade 8 Math Proficiency,Percent English Language Learners,Percent Students with Disabilities,Economic Need Index,Percent in Temp Housing,...,BN,State & Local Funding (M),Federal Funding (M),Total Funding (M),Borough,N count - Economic Need Index,White,Hispanic,Black,Asian
0,01M292,Orchard Collegiate Academy,High School,197.0,2.701,2.325,0.117,0.289,0.871,0.183,...,M292,4.138736,0.155015,4.293751,Manhattan,171.587,5.91,112.093,52.993,18.912


# Imputing NaN Values
After merging the datasets dataframe, we noticed there were some **NaN** values present in our dataframe. In particular, out of the 486 New York City public high schools, **65 school's had missing funding data.**

Due to the mean being greater than the median, we can intuitively figure out that the distribution is positively skewed. Since the data is skewed, we imputed the NaN values with the median for a better representation of the data.

Given the 13% of missing Funding Data** from the schools compared with the Funding information for other schools as well as the distriubtion of the Funding data, we decided to impute the NaN values with the median for "Total Funding" to account for any outliers in for School Funding. In addition, we imputed the NaN values with the mean for "Total Funding" per Student, which provides better accuracy for our project.

In [11]:
# finding only the columns that have NaN values
print('Columns with NaN values')
print()
for columns in df.columns:
    if df[columns].isnull().any() == True:
        print('[',columns,']')
        print('     NaN count:', df[columns].isnull().sum(axis = 0))
        print('-------------------------------------------')

Columns with NaN values

[ State & Local Funding (M) ]
     NaN count: 65
-------------------------------------------
[ Federal Funding (M) ]
     NaN count: 65
-------------------------------------------
[ Total Funding (M) ]
     NaN count: 65
-------------------------------------------


In [12]:
# changing funding columns into floats
df = df.astype({'State & Local Funding (M)': 'float64',
                'Federal Funding (M)': 'float64',
                'Total Funding (M)': 'float64'
              })
pd.options.display.float_format = "{:.3f}".format

# basic statistics
df[['State & Local Funding (M)',
    'Federal Funding (M)',
    'Total Funding (M)']].describe()

,State & Local Funding (M),Federal Funding (M),Total Funding (M)
count,421.000,421.000,421.000
mean,9.625,0.537,10.162
std,8.622,0.535,9.044
min,2.528,0.002,2.673
25%,5.526,0.265,5.837
50%,6.772,0.405,7.194
75%,8.978,0.575,9.582
max,57.611,4.113,58.096


In [13]:
# Imputing NaN values with the median
df['State & Local Funding (M)'].fillna((df['State & Local Funding (M)'].median()), inplace=True)
df['Federal Funding (M)'].fillna((df['Federal Funding (M)'].median()), inplace=True)
df['Total Funding (M)'].fillna((df['Total Funding (M)'].median()), inplace=True)

df[['State & Local Funding (M)','Federal Funding (M)',
    'Total Funding (M)']].tail(2)

,State & Local Funding (M),Federal Funding (M),Total Funding (M)
484,6.772,0.405,7.194
485,6.772,0.405,7.194


# Imputing "N<15" Data

In [14]:
# replace values with the mean
for columns in df.columns:
     if len(df[df[columns] == 'N<15']):
        print('N<15 count: ', df[columns].str.count('N<15').sum())
        # replacing "N<15" values with NaN
        df[columns] = df[columns].replace('N<15', np.NaN)
        print('average:',df[columns].mean())
        # imputing NaN with the mean.
        df[columns] = df[columns].fillna(df[columns].mean())
        print(df[columns].value_counts().head(3))
        print('------------------------------------------------------------')

N<15 count:  21.0
average: 0.8360559139784945
0.836    21
1.000    20
0.800     7
Name: Metric Value - 4-Year Graduation Rate - All Students, dtype: int64
------------------------------------------------------------
N<15 count:  8.0
average: 73.3673640167364
73.367    8
67.900    6
67.100    6
Name: Metric Value - Average Regents Score - English (Common Core), dtype: int64
------------------------------------------------------------
N<15 count:  30.0
average: 66.48991228070174
66.490    30
62.700     8
60.600     7
Name: Metric Value - Average Regents Score - Algebra I (Common Core), dtype: int64
------------------------------------------------------------
N<15 count:  21.0
average: 0.48484946236559134
0.485    21
1.000    11
0.264     4
Name: Metric Value - College and Career Preparatory Course Index, dtype: int64
------------------------------------------------------------
N<15 count:  21.0
average: 0.564810752688172
0.565    21
0.500     8
0.333     7
Name: Metric Value - 4-Year Col

# Imputing "." Values

In [15]:
# replace values with the mean
for columns in df.columns:
     if len(df[df[columns] == '.']):
        print('"." count: ', df[columns].str.count('.').sum())
        # replacing "N<15" values with NaN
        df[columns] = df[columns].replace('.', np.NaN)
        print('average:',df[columns].mean())
        # imputing NaN with the mean.
        df[columns] = df[columns].fillna(df[columns].mean())
        print(df[columns].value_counts().head(3))
        print('------------------------------------------------------------')

"." count:  66.0
average: 6.735476190476191
6.735    66
6.100    16
7.900    15
Name: Years of principal experience at this school, dtype: int64
------------------------------------------------------------
"." count:  65.0
average: 0.7534513064133016
0.753    65
0.750    11
0.800     8
Name: Percent of teachers with 3 or more years of experience, dtype: int64
------------------------------------------------------------
"." count:  9.0
average: 0.8749371069182389
0.875    9
0.911    8
0.849    8
Name: Student Attendance Rate, dtype: int64
------------------------------------------------------------
"." count:  10.0
average: 0.3579579831932773
0.358    10
0.513     6
0.472     5
Name: Percent of Students Chronically Absent, dtype: int64
------------------------------------------------------------
"." count:  65.0
average: 0.9663016627078385
0.966    65
0.967    29
0.965    22
Name: Teacher Attendance Rate, dtype: int64
------------------------------------------------------------
"." coun

In [16]:
# renaming Columns
df.rename(columns = {
'Economic Need Index':'Economic Need Index (%)',
'Metric Value - 4-Year Graduation Rate - All Students':'Graduation Rate (%)',
'Metric Value - 4-Year College Readiness Index':'College Readiness Index (%)',  
'Metric Value - College and Career Preparatory Course Index':'College & Career Preparatory Course Index (%)',       
'Metric Value - Average Regents Score - English (Common Core)':'Avg Regents Score - English (%)',   
'Metric Value - Average Regents Score - Algebra I (Common Core)': 'Avg Regents Score - Algebra I (%)',         
'Percent Asian':'Asian (%)','Percent Black':'Black (%)','Percent Hispanic':'Hispanic (%)','Percent White':'White (%)',
'N count - Economic Need Index': 'Economic Need Index',
'N count - 4-Year Graduation Rate - All Students':'Students Graduated',
'N count - 4-Year College Readiness Index': 'College Readiness Index',
'N count - College and Career Preparatory Course Index': 'College & Career Preparatory Course Index',
'Years of principal experience at this school':'Years of principal experience', 
'Average Grade 8 English Proficiency':'Avg Grade 8 English Proficiency',
'Average Grade 8 Math Proficiency':'Avg Grade 8 Math Proficiency',    
'Teacher Attendance Rate':'Teacher Attendance (%)',
'Student Attendance Rate':'Student Attendance (%)',
'Percentage of Students with 90%+ Attendance': 'Students with 90%+ Attendance (%)',
'Percent English Language Learners': 'English Language Learners (%)',
'Percent Students with Disabilities': 'Students with Disabilities (%)',
'Percent in Temp Housing':'Temp Housing (%)',
'Percent HRA Eligible': 'HRA Eligible (%)',
'Percent of teachers with 3 or more years of experience':'Teachers w/ 3+ years of experience (%)',
'Percent of Students Chronically Absent': 'Students Chronically Absent (%)'},inplace = True)

In [17]:
# creating a classifying feature for schools graduation rates
grad = []  
for value in df['Graduation Rate (%)']:
    if value >= .95:
        grad.append('High Graduation')
    elif value <= 0.80:
        grad.append('Low Graduation')
    else:
        grad.append('Medium Graduation')
df['Graduation'] = grad 
df['Graduation'].value_counts().unique

<bound method Series.unique of Medium Graduation    215
Low Graduation       172
High Graduation       99
Name: Graduation, dtype: int64>

In [18]:
# reordering all columns
df = df[['DBN','BN','School Name','School Type','Borough','Enrollment',
'State & Local Funding (M)','Federal Funding (M)','Total Funding (M)',
'Economic Need Index (%)',
'Graduation Rate (%)',
'College Readiness Index (%)',  
'College & Career Preparatory Course Index (%)',       
'Avg Regents Score - English (%)',    
'Avg Regents Score - Algebra I (%)',
'Asian (%)','Black (%)','Hispanic (%)','White (%)',         
'Students Graduated',
'College Readiness Index',
'College & Career Preparatory Course Index',
'Economic Need Index',
'White','Hispanic','Black','Asian',
'Years of principal experience',
'Avg Grade 8 English Proficiency',
'Avg Grade 8 Math Proficiency',
'Teacher Attendance (%)','Student Attendance (%)',
'Students with 90%+ Attendance (%)',         
'English Language Learners (%)',
'Students with Disabilities (%)',
'Temp Housing (%)',
'HRA Eligible (%)',
'Teachers w/ 3+ years of experience (%)',
'Students Chronically Absent (%)',
'Graduation']]

# Establishing Dtype

In [19]:
# rounding and changing data types to INT
int_vars =['Enrollment','Students Graduated','College Readiness Index',                          
'College & Career Preparatory Course Index','White','Hispanic',
'Black','Asian','Economic Need Index']
for column in df[int_vars].columns:
    df[column] = df[column].round(decimals=0)
    df[column] = df[column].astype(int)   
# change borough data type as category
df['Borough'] = df['Borough'].astype('category')

In [20]:
# round all floats
for col in df.columns:
    if df[col].dtype == 'float':
        df[col] = df[col].round(3)

# Completion of Data Cleaning

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 0 to 485
Data columns (total 40 columns):
 #   Column                                         Non-Null Count  Dtype   
---  ------                                         --------------  -----   
 0   DBN                                            486 non-null    object  
 1   BN                                             486 non-null    object  
 2   School Name                                    486 non-null    object  
 3   School Type                                    486 non-null    object  
 4   Borough                                        486 non-null    category
 5   Enrollment                                     486 non-null    int64   
 6   State & Local Funding (M)                      486 non-null    float64 
 7   Federal Funding (M)                            486 non-null    float64 
 8   Total Funding (M)                              486 non-null    float64 
 9   Economic Need Index (%)                    

In [22]:
# save clean data frame to csv file
df.to_csv('data/clean_data.csv',index=False, header=True)

NYC Geography Question
In what way can we see location visualized in the data?
Using DBN to group high schools by borough
Then use DBN to focus on the schools by district
How does the graduation rate differ among racial categories?
How is “average regents complete rate” distributed across racial categories?

# Merging completed dataframe with school locations data

 Source: https://www.kaggle.com/new-york-city/ny-2010-2016-school-safety-report/home

In [23]:
geo_csv = pd.read_csv('data/2010-2016-School-Safety-Report.csv', header=0, 
                           usecols=[0,2,3,5,25,26,27,33])                  
geo_csv

,School Year,DBN,Location Name,Address,Postcode,Latitude,Longitude,NTA
0,2013-14,15K001,P.S. 001 The Bergen,309 47 STREET,11220.000,40.649,-74.012,Sunset Park West ...
1,2013-14,17K002,Parkside Preparatory Academy,655 PARKSIDE AVENUE,11226.000,40.656,-73.952,Prospect Lefferts Gardens-Wingate ...
2,2013-14,75K141,P.S. K141,655 PARKSIDE AVENUE,11226.000,40.656,-73.952,Prospect Lefferts Gardens-Wingate ...
3,2013-14,84K704,Explore Charter School,655 PARKSIDE AVENUE,11226.000,40.656,-73.952,Prospect Lefferts Gardens-Wingate ...
4,2013-14,NaN,655 PARKSIDE AVENUE CONSOLIDATED LOCATION,655 PARKSIDE AVENUE,11226.000,40.656,-73.952,Prospect Lefferts Gardens-Wingate ...
...,...,...,...,...,...,...,...,...
6305,2015-16,08X519,Felisa Rincon de Gautier Institute for Law and...,1440 STORY AVENUE,10473.000,40.821,-73.881,Soundview-Castle Hill-Clason Point-Harding Par...
6306,2015-16,08X537,Bronx Arena High School,1440 STORY AVENUE,10473.000,40.821,-73.881,Soundview-Castle Hill-Clason Point-Harding Par...
6307,2015-16,NaN,1440 STORY AVENUE CONSOLIDATED LOCATION,1440 STORY AVENUE,10473.000,40.821,-73.881,Soundview-Castle Hill-Clason Point-Harding Par...
6308,2015-16,12X271,East Bronx Academy for the Future,1716 SOUTHERN BOULEVARD,10460.000,40.836,-73.888,Crotona Park East ...


In [24]:
# merging quality report and locations data
geo_df = pd.merge(df, geo_csv, on='DBN', how='left')
# creating a new dataframe with only the most recent years
geo_df = geo_df[geo_df['School Year'].astype(str).str.contains('2015-16')]
# seeing how many rows have missing location data
print('Rows with missing location data:', geo_df.isnull().any(axis = 1).sum())
print()
print('Original Completed Data frame:', df.shape)
print('Original Completed Data frame with Geo Attributes', geo_df.shape)

geo_df.dropna(axis='rows',inplace=True)
geo_df.drop('School Year',axis = 1, inplace = True)
geo_df['Postcode'] = geo_df['Postcode'].astype('int64')

geo_df.head(2)

Rows with missing location data: 1

Original Completed Data frame: (486, 40)
Original Completed Data frame with Geo Attributes (437, 47)


,DBN,BN,School Name,School Type,Borough,Enrollment,State & Local Funding (M),Federal Funding (M),Total Funding (M),Economic Need Index (%),...,HRA Eligible (%),Teachers w/ 3+ years of experience (%),Students Chronically Absent (%),Graduation,Location Name,Address,Postcode,Latitude,Longitude,NTA
2,01M292,M292,Orchard Collegiate Academy,High School,Manhattan,197,4.139,0.155,4.294,0.871,...,0.782,0.750,0.289,Medium Graduation,Henry Street School for International Studies,220 HENRY STREET,10002,40.714,-73.986,Lower East Side ...
5,01M448,M448,University Neighborhood High School,High School,Manhattan,483,6.908,0.548,7.456,0.830,...,0.723,0.600,0.274,Medium Graduation,University Neighborhood High School,200 MONROE STREET,10002,40.712,-73.984,Lower East Side ...


In [25]:
#save clean data frame (with geo data) to csv
geo_df.to_csv('data/clean_geo_data.csv',index=False, header=True)